In [13]:
#importing libraries
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
import uuid
import pinecone


In [14]:
#extract data from pdf

loader = DirectoryLoader(
    path='C:\End to End Projects\Medical Chatbot Gen AI\data',
    glob='*.pdf',
    loader_cls=PyPDFLoader
    )
documents = loader.load()

In [15]:
#creating text chunks

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)

text_chunks = text_splitter.split_documents(documents)

In [16]:
len(text_chunks)

7020

In [17]:
#embedding model

embedding = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [18]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [19]:
query = embedding.embed_query('Hello')
print(len(query))

384


In [20]:
PINECONE_API_KEY = "aca07530-5144-453c-ac16-318eebed1265"
PINECONE_API_ENV = "gcp-starter"

In [21]:
#initializing pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)


In [22]:
index_name="medical-chatbot"

In [23]:
#Creating Embeddings for Each of The Text Chunks & storing
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],
                                embedding,
                                index_name=index_name)

In [28]:
#If we already have an index we can load it like this
docsearch = Pinecone.from_existing_index(index_name,embedding)

query = "What are Allergies"
docs = docsearch.similarity_search(query,k=3)

print(docs)

[Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commo

In [32]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [33]:
PROMPT = PromptTemplate(template=prompt_template,input_variables=['context','question'])
chain_type_kwargs = {'prompt':PROMPT}

In [37]:
llm=CTransformers(model="C:\End to End Projects\Medical Chatbot Gen AI\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [39]:
qa = RetrievalQA.from_chain_type(llm=llm,
                            chain_type='stuff',
                            retriever=docsearch.as_retriever(search_kwargs={'k':2}),
                            return_source_documents = True,
                            chain_type_kwargs = chain_type_kwargs)
                                           

In [40]:
user_input = input(f'Input Prompt:')
result = qa({'query':user_input})
print(result)

{'query': 'what is Acne?', 'result': 'Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.', 'source_documents': [Document(page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is', metadata={}), Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25', metadata={})]}


In [41]:
user_input = input(f'Input Prompt:')
result = qa({'query':user_input})
print(result)

{'query': 'what is chalasia?', 'result': 'Chalasia is not a term used in the context of smooth muscle contraction or digestion. It is not a medical term, and I could not find any information on it through my research.', 'source_documents': [Document(page_content='smooth muscle contraction that forces foodthrough the digestive tract.\nReflux —An abnormal backward or return flow of\na fluid.Description\nAchondroplasia is one of a number of chondodystro-\nphies, in which the development of cartilage, and there-fore, bone is disturbed. The disorder appears in approxi-mately one in every 10,000 births. Achondroplasia isusually diagnosed at birth, owing to the characteristicappearance of the newborn.\nNormal bone growth depends on the production of', metadata={}), Document(page_content='worm ), which occurs on the skin or candidiasis (a yeast\ninfection, also known as trush), which can occur in thethroat, in the vagina, or in other parts of the body. They arealso used to treat other deep fun